<a href="https://colab.research.google.com/github/doduck210/Instrument_Classifier/blob/main/instrument_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#for using CoLab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#for using CoLab
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

ERROR: torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [ ]:
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import random
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
#데이터 확인
npz=np.load('/content/gdrive/My Drive/audio/cqt.npz')
x=npz['spec']
y=npz['instr']
print("input shape : ",x.shape) 
print("label shape : ",y.shape)

input shape :  (20488, 168, 87)
label shape :  (20488, 2)


In [ ]:
use_cuda = torch.cuda.is_available()
class InstrumentClassifier(nn.Module):
  def __init__(self):
    super(InstrumentClassifier,self).__init__()
    #input : [256, 1, 168, 87]
    conv1=nn.Conv2d(1,6,(3,3)) #6@166x85
    #ReLU
    pool1=nn.MaxPool2d(2) #6@83x42
    conv2=nn.Conv2d(6,10,(3,3)) #10@81x40
    #ReLU
    pool2=nn.MaxPool2d(2) #10@40x20
    conv3=nn.Conv2d(10,32,(23,3)) #28@18x18
    #ReLU
    pool3=nn.MaxPool2d(2) #28@9x9
    #onefilter=nn.Conv2d(28,1,(1,1)) 
    dropout= nn.Dropout()

    self.conv_module=nn.Sequential(
        conv1,
        nn.ReLU(),
        pool1,
        conv2,
        nn.ReLU(),
        pool2,
        conv3,
        nn.ReLU(), dropout,
        pool3
        #onefilter
    )

    fc1=nn.Linear(32*81,174)
    #ReLU
    #fc2=nn.Linear(500,174)

    #nn.init.xavier_uniform_(fc1.weight)
    #nn.init.xavier_uniform_(fc2.weight)
    #nn.init.xavier_uniform_(fc3.weight)

    self.fc_module=nn.Sequential(
        fc1,
        #nn.ReLU()
        #fc2
    )
    if use_cuda:
      self.conv_module=self.conv_module.cuda()
      self.fc_module=self.fc_module.cuda()
    
  def forward(self,x):
    out=self.conv_module(x)
    dim=1
    out=out.view(out.size(0),-1)
    out=self.fc_module(out)
    return F.softmax(out,dim=1)

In [ ]:
cnn=InstrumentClassifier()
criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(cnn.parameters(),lr=1e-3)

In [ ]:
ridx = list(range(len(x)))
random.shuffle(ridx)
batch_size = 256
for e in range(100):
  for b in range(0,len(ridx),batch_size) : 
    tr=ridx[b:b+batch_size]
    input=torch.tensor(x[tr])
    input=input.unsqueeze(1)
    input=input.cuda()
    output=torch.tensor(y[tr,0])
    output=torch.tensor(output,dtype=torch.long)
    output=output.cuda()
    #print("x : ",input.shape) #torch.Size([256, 1, 168, 87])
    #print("y : ",output.shape) #torch.Size([256])

    optimizer.zero_grad()
    model_output=cnn(input) #torch.Size([256, 174])
    #print(model_output.shape)
    loss=criterion(model_output,output)
    loss.backward()
    optimizer.step()
    print('e : ',e,"loss : ",loss)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


e :  0 loss :  tensor(5.1590, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1589, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1587, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1578, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1581, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1563, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1573, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1577, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1596, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1565, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1574, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1568, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1594, device='cuda:0', grad_fn=<NllLossBackward>)
e :  0 loss :  tensor(5.1573, device='

KeyboardInterrupt: ignored